<a href="https://colab.research.google.com/github/Hammadtahir7/Card-Fraud-Detection/blob/master/CCFD_1DCNN_ECD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [0]:
import io
import pandas as pd
df2 = pd.read_csv(io.BytesIO(uploaded['ECD.csv']),header=None)

In [4]:
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,162183.0,2.049094,0.186189,-1.707198,0.530768,0.160589,-1.448570,0.239310,-0.353611,0.634425,-0.471390,-0.349233,-0.073474,-0.303130,-0.664254,0.977499,0.157691,0.469860,0.446847,-0.502306,-0.232666,0.197782,0.741141,-0.009744,-0.085057,0.228384,-0.097292,-0.001028,-0.032390,2.99,0.0
1,120967.0,2.125540,-0.030714,-1.527653,0.121046,0.543172,-0.347988,0.157221,-0.229126,0.477999,0.046959,-1.314079,0.508379,0.891406,0.081326,0.065139,0.109356,-0.623472,-0.671603,0.435948,-0.125804,-0.336497,-0.838932,0.275173,0.049145,-0.156765,0.205919,-0.072321,-0.059009,1.98,0.0
2,26484.0,-4.155859,-5.705748,0.274699,-0.993262,-6.059393,5.210848,5.811316,0.367888,1.750710,-4.171672,-0.583742,0.346628,-0.719006,-0.899347,0.006643,-1.784000,1.457844,-0.952985,-1.315135,3.944592,1.371671,1.195815,4.188762,-1.091077,1.033044,0.224493,-0.486741,0.194275,1937.66,0.0
3,65751.0,-0.566420,-0.579576,0.823503,-1.451240,-0.583587,0.206381,1.601392,-0.370446,-1.910354,0.252497,0.922508,-0.114149,1.008346,-0.282106,-0.246635,1.345463,-0.401365,-0.993434,1.474268,0.977739,-0.065082,-0.761357,0.641524,-0.568974,-0.053164,-0.690995,-0.228630,-0.157254,320.05,0.0
4,137025.0,0.060858,-0.261762,-1.699493,-1.202327,3.699527,3.196249,0.437208,0.421541,0.492435,-0.455964,0.028435,-0.329083,-0.552349,-1.568433,-0.440477,-0.296621,0.481656,-0.061843,-0.387865,-0.014550,0.008303,0.534602,0.089602,0.667918,0.017798,0.611584,-0.469946,-0.514370,11.50,0.0


In [0]:
features =df2.drop(df2.columns[30],axis=1)
labels=df2[df2.columns[30]]

In [5]:
import numpy as np
unique_elements, counts_elements = np.unique(labels, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[0.00e+00 1.00e+00]
 [4.92e+03 4.92e+02]]


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
features = scaler.fit_transform(features)
print(features[0])

X_train, X_test, y_train, y_test=train_test_split(features,labels,test_size=0.2, random_state=42)

#X_train=X_train.astype('float32')
#X_test=X_test.astype('float32')
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

[ 1.45094259  0.80223991 -0.06502102 -0.33078773  0.04703112  0.20079889
 -0.93863499  0.25309466 -0.18275693  0.58255806  0.02030165 -0.4191561
  0.19984054 -0.28898097 -0.01702708  1.08823192  0.28239682  0.35878101
  0.47503238 -0.63500995 -0.34422048  0.10376635  0.9152007  -0.00556151
 -0.14503745  0.42723258 -0.21816581 -0.02726437 -0.13146827 -0.37888298]
(4329, 30)
(1083, 30)
(4329,)
(1083,)


In [7]:
X_train=X_train.reshape(4329,1,30)
X_test=X_test.reshape(1083,1,30)
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [8]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [9]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(1,30)))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [10]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_m,precision_m, recall_m])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
#train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)




Train on 4329 samples, validate on 1083 samples
Epoch 1/10





4329/4329 [==============================] - 1s 312us/step - loss: 0.1884 - acc: 0.9513 - f1_m: 0.5969 - precision_m: 0.7172 - recall_m: 0.5549 - val_loss: 0.0799 - val_acc: 0.9843 - val_f1_m: 0.8156 - val_precision_m: 0.8966 - val_recall_m: 0.7809
Epoch 2/10
4329/4329 [==============================] - 0s 78us/step - loss: 0.0639 - acc: 0.9848 - f1_m: 0.8571 - precision_m: 0.9250 - recall_m: 0.8247 - val_loss: 0.0825 - val_acc: 0.9843 - val_f1_m: 0.8156 - val_precision_m: 0.8966 - val_recall_m: 0.7809
Epoch 3/10
4329/4329 [==============================] - 0s 76us/step - loss: 0.0596 - acc: 0.9861 - f1_m: 0.8493 - precision_m: 0.9038 - recall_m: 0.8230 - val_loss: 0.0804 - val_acc: 0.9843 - val_f1_m: 0.8174 - val_precision_m: 0.8744 - val_recall_m: 0.8030
Epoch 4/10
4329/4329 [==============================] - 0s 73us/step - loss: 0.0555 - acc: 0.9866 - f1_m: 0.8462 - precision_m: 0.8950 - recall_m: 0.8196 - val_loss: 

In [15]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)


print("loss: "+"{:.2%}".format(loss));
print("acc: "+"{:.2%}".format(accuracy));
print("f1: "+"{:.2%}".format(f1_score));
print("precision: "+"{:.2%}".format(precision));
print("recall: "+"{:.2%}".format(recall));

loss: 7.98%
acc: 98.71%
f1: 86.09%
precision: 91.87%
recall: 83.85%


In [0]:
y_pred=model.predict_classes(X_test)

In [17]:
from sklearn import metrics
print("Accuracy: "+"{:.2%}".format(metrics.accuracy_score(y_test, y_pred)));
print("Precision: "+"{:.2%}".format(metrics.precision_score(y_test, y_pred)));
print("Recall: "+"{:.2%}".format(metrics.recall_score(y_test, y_pred)));
print("F1: "+"{:.2%}".format(metrics.f1_score(y_test,y_pred)));
print(metrics.confusion_matrix(y_test,y_pred));

Accuracy: 98.71%
Precision: 97.59%
Recall: 87.10%
F1: 92.05%
[[988   2]
 [ 12  81]]


In [17]:
for i in range(len(y_test)):
  if(y_test[i]==1):
    print(i)
    print(y_test[i])
    print(y_pred[i])
    print("-----")

19
1.0
1
-----
21
1.0
1
-----
23
1.0
1
-----
31
1.0
1
-----
38
1.0
1
-----
39
1.0
1
-----
58
1.0
1
-----
60
1.0
1
-----
65
1.0
1
-----
68
1.0
1
-----
71
1.0
0
-----
76
1.0
1
-----
84
1.0
0
-----
101
1.0
1
-----
118
1.0
1
-----
130
1.0
1
-----
134
1.0
1
-----
149
1.0
1
-----
153
1.0
1
-----
156
1.0
0
-----
191
1.0
1
-----
203
1.0
0
-----
210
1.0
1
-----
215
1.0
1
-----
228
1.0
0
-----
248
1.0
1
-----
253
1.0
0
-----
268
1.0
1
-----
274
1.0
1
-----
276
1.0
1
-----
289
1.0
0
-----
305
1.0
1
-----
335
1.0
1
-----
340
1.0
1
-----
345
1.0
0
-----
348
1.0
1
-----
349
1.0
0
-----
387
1.0
1
-----
422
1.0
1
-----
427
1.0
1
-----
431
1.0
1
-----
435
1.0
1
-----
453
1.0
1
-----
483
1.0
0
-----
488
1.0
1
-----
499
1.0
1
-----
510
1.0
1
-----
525
1.0
1
-----
538
1.0
1
-----
545
1.0
0
-----
552
1.0
1
-----
554
1.0
1
-----
559
1.0
1
-----
616
1.0
1
-----
617
1.0
1
-----
626
1.0
1
-----
630
1.0
1
-----
650
1.0
1
-----
667
1.0
1
-----
675
1.0
1
-----
685
1.0
1
-----
687
1.0
1
-----
691
1.0
1
-----
713
1